In [24]:
import re
import pandas as pd # CSV file I/O (pd.read_csv)
from nltk.corpus import stopwords

import copy
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [25]:
news = pd.read_excel("C:\\Users\\Kratika\\Downloads\\News333.xlsx")
news.head()

,Title,Body,Category,MegaCategory
0,BS-VI transition may lead to dumping of old st...,BS-VI transition may lead to dumping of old st...,Annual Report - Comments made in the Annual Re...,Financial
1,Annual Report 2016-2017 of Bajaj Finserv Limited,It is a broadly described annual report of Baj...,Annual Report - Comments made in the Annual Re...,Financial
2,Annual Report 2017-2018 of Eicher Motors Limit...,It is a broadly described annual report of\nEi...,Annual Report - Comments made in the Annual Re...,Financial
3,Hindalco Industries : Chairman Kumar Mangalam ...,(You can enter multiple email addresses separa...,Annual Report - Comments made in the Annual Re...,Financial
4,"WIPRO continues it's uptrend, although on a we...","WIPRO continues it's uptrend, although on a we...",Annual Report - Comments made in the Annual Re...,Financial


In [26]:
from nltk.tokenize import word_tokenize 

def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    stop_words.union(['january','february','march','april','may','june','july','august','september','october','november','december'])
    stop_words.union(['jan','feb','mar','apr','may','jun','jul','aug','sept','oct','nov','dec'])
    stop_words.union(['monday', 'tuesday', 'wednesday', 'thursday','friday','saturday','sunday'])
    stop_words.union(['pm', 'am'])
    word_tokens = word_tokenize(data) 
    filtered_sentence = []
    
    for w in word_tokens: 
        if w.lower() not in stop_words: 
            filtered_sentence.append(w) 
    
    return " ".join(filtered_sentence)

In [27]:

news['clean_title'] = news['Title'].apply(remove_stop_words) 

#remove_stop_words_Body


In [28]:


news['clean_body'] = news['Body'].apply(remove_stop_words) 


In [29]:
news.head()

,Title,Body,Category,MegaCategory,clean_title,clean_body
0,BS-VI transition may lead to dumping of old st...,BS-VI transition may lead to dumping of old st...,Annual Report - Comments made in the Annual Re...,Financial,BS-VI transition may lead dumping old stock : ...,BS-VI transition may lead dumping old stock : ...
1,Annual Report 2016-2017 of Bajaj Finserv Limited,It is a broadly described annual report of Baj...,Annual Report - Comments made in the Annual Re...,Financial,Annual Report 2016-2017 Bajaj Finserv Limited,broadly described annual report Bajaj Finserv ...
2,Annual Report 2017-2018 of Eicher Motors Limit...,It is a broadly described annual report of\nEi...,Annual Report - Comments made in the Annual Re...,Financial,Annual Report 2017-2018 Eicher Motors Limited ...,broadly described annual report Eicher Motors ...
3,Hindalco Industries : Chairman Kumar Mangalam ...,(You can enter multiple email addresses separa...,Annual Report - Comments made in the Annual Re...,Financial,Hindalco Industries : Chairman Kumar Mangalam ...,( enter multiple email addresses separated com...
4,"WIPRO continues it's uptrend, although on a we...","WIPRO continues it's uptrend, although on a we...",Annual Report - Comments made in the Annual Re...,Financial,"WIPRO continues 's uptrend , although weaker note","WIPRO continues 's uptrend , although weaker n..."


In [30]:
# ## Split data
print("\nSplitting data")
title_tr, title_te, MegaCategory_tr, MegaCategory_te = train_test_split(news['clean_body'], news.MegaCategory,test_size =.02)
title_tr, title_de, MegaCategory_tr , MegaCategory_de = train_test_split(title_tr,MegaCategory_tr,test_size =.02)
print("Training: ",len(title_tr))
print("Developement: ",len(title_de),)
print("Testing: ",len(title_te))


Splitting data
Training:  2379
Developement:  49
Testing:  50


In [31]:
from sklearn.preprocessing import LabelEncoder

# # Data Preprocessing
# ## Vectorization of data
# Vectorize the data using Bag of words (BOW)
print("\nVectorizing data")
tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
stop_words = nltk.corpus.stopwords.words("english")
vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize, stop_words=stop_words)

vectorizer.fit(iter(title_tr))
Xtr = vectorizer.transform(iter(title_tr))
Xde = vectorizer.transform(iter(title_de))
Xte = vectorizer.transform(iter(title_te))

encoder = LabelEncoder()
encoder.fit(MegaCategory_tr)
Ytr = encoder.transform(MegaCategory_tr)
Yde = encoder.transform(MegaCategory_de)
Yte = encoder.transform(MegaCategory_te)


Vectorizing data


In [32]:
print(Ytr)

[11  4  8 ... 11 11  5]


In [33]:
Xtr

<2379x16524 sparse matrix of type '<class 'numpy.int64'>'
	with 106268 stored elements in Compressed Sparse Row format>

In [34]:
# # Train Models
# ### Baseline Model
# “stratified”: generates predictions by respecting the training set’s class distribution.
print("\n\nTraining baseline classifier")
dc = DummyClassifier(strategy="stratified")
dc.fit(Xtr, Ytr)
pred = dc.predict(Xde)



Training baseline classifier


In [35]:
print(classification_report(Yde, pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred == Yde)))


                      precision    recall  f1-score   support

Awards & Recognition       0.47      0.33      0.39        27
   Corporate Updates       0.00      0.00      0.00         6
               Event       0.00      0.00      0.00         4
              Events       0.00      0.00      0.00         2
           Financial       0.47      0.33      0.39        27
              Growth       0.47      0.33      0.39        27
       Industry News       0.47      0.33      0.39        27
         Market Buzz       0.47      0.33      0.39        27
       Negative News       0.47      0.33      0.39        27
              People       0.00      0.00      0.00         1
   Stock Performance       0.10      0.20      0.13         5
  Uncategorised News       0.00      0.00      0.00         2

           micro avg       0.34      0.29      0.31     31952
           macro avg       0.21      0.17      0.18     31952
        weighted avg       0.40      0.29      0.33     31952

Accur

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2379, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [36]:

# ### Decision Tree
print("Training Decision tree")
dt = DecisionTreeClassifier()
dt.fit(Xtr, Ytr)
pred = dt.predict(Xde)
print(classification_report(Yde, pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred == Yde)))

Training Decision tree
                      precision    recall  f1-score   support

Awards & Recognition       0.71      0.74      0.73        27
   Corporate Updates       1.00      0.17      0.29         6
               Event       0.00      0.00      0.00         4
              Events       0.50      0.50      0.50         2
           Financial       0.71      0.74      0.73        27
              Growth       0.71      0.74      0.73        27
       Industry News       0.71      0.74      0.73        27
         Market Buzz       0.71      0.74      0.73        27
       Negative News       0.71      0.74      0.73        27
              People       0.50      1.00      0.67         1
   Stock Performance       0.29      0.40      0.33         5
  Uncategorised News       0.50      0.50      0.50         2

           micro avg       0.63      0.65      0.64     31952
           macro avg       0.48      0.45      0.43     31952
        weighted avg       0.66      0.65    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2379, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [37]:
# ### Random Forest
print("Training Random Forest")
rf = RandomForestClassifier(n_estimators=40)
rf.fit(Xtr, Ytr)
pred = rf.predict(Xde)
print(classification_report(Yde, pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred == Yde)))

Training Random Forest
                      precision    recall  f1-score   support

Awards & Recognition       0.71      0.89      0.79        27
   Corporate Updates       0.67      0.33      0.44         6
               Event       0.00      0.00      0.00         4
              Events       0.00      0.00      0.00         2
           Financial       0.71      0.89      0.79        27
              Growth       0.71      0.89      0.79        27
       Industry News       0.71      0.89      0.79        27
         Market Buzz       0.71      0.89      0.79        27
       Negative News       0.71      0.89      0.79        27
              People       1.00      1.00      1.00         1
   Stock Performance       0.50      0.80      0.62         5
  Uncategorised News       0.00      0.00      0.00         2

           micro avg       0.67      0.80      0.73     31952
           macro avg       0.48      0.57      0.51     31952
        weighted avg       0.65      0.80    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2379, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [38]:

# ### Multinomial Naive Bayesian
print("Training Multinomial Naive Bayesian")
nb = MultinomialNB()
nb.fit(Xtr, Ytr)
pred_nb = nb.predict(Xde)
print(classification_report(Yde, pred_nb, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred == Yde)))

Training Multinomial Naive Bayesian
                      precision    recall  f1-score   support

Awards & Recognition       0.79      0.85      0.82        27
   Corporate Updates       0.67      0.33      0.44         6
               Event       0.00      0.00      0.00         4
              Events       1.00      0.50      0.67         2
           Financial       0.79      0.85      0.82        27
              Growth       0.79      0.85      0.82        27
       Industry News       0.79      0.85      0.82        27
         Market Buzz       0.79      0.85      0.82        27
       Negative News       0.79      0.85      0.82        27
              People       1.00      1.00      1.00         1
   Stock Performance       0.31      0.80      0.44         5
  Uncategorised News       1.00      0.50      0.67         2

           micro avg       0.71      0.77      0.74     31952
           macro avg       0.53      0.58      0.53     31952
        weighted avg       0.71 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2379, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [39]:
# ### Support Vector Classification
print("Training Support Vector Classification")
from sklearn.svm import SVC
svc = SVC()
svc.fit(Xtr, Ytr)
pred = svc.predict(Xde)
print(classification_report(Yde, pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred == Yde)))

Training Support Vector Classification


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


                      precision    recall  f1-score   support

Awards & Recognition       0.55      1.00      0.71        27
   Corporate Updates       0.00      0.00      0.00         6
               Event       0.00      0.00      0.00         4
              Events       0.00      0.00      0.00         2
           Financial       0.55      1.00      0.71        27
              Growth       0.55      1.00      0.71        27
       Industry News       0.55      1.00      0.71        27
         Market Buzz       0.55      1.00      0.71        27
       Negative News       0.55      1.00      0.71        27
              People       0.00      0.00      0.00         1
   Stock Performance       0.00      0.00      0.00         5
  Uncategorised News       0.00      0.00      0.00         2

           micro avg       0.55      0.83      0.66     31952
           macro avg       0.23      0.41      0.29     31952
        weighted avg       0.46      0.83      0.59     31952

Accur

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2379, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [40]:
# ### Multilayered Perceptron
print("Training Multilayered Perceptron")
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1, max_iter=400)
mlp.fit(Xtr, Ytr)
pred = mlp.predict(Xde)
print(classification_report(Yde, pred, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred == Yde)))

Training Multilayered Perceptron
                      precision    recall  f1-score   support

Awards & Recognition       0.81      0.93      0.86        27
   Corporate Updates       1.00      0.33      0.50         6
               Event       0.00      0.00      0.00         4
              Events       0.50      0.50      0.50         2
           Financial       0.81      0.93      0.86        27
              Growth       0.81      0.93      0.86        27
       Industry News       0.81      0.93      0.86        27
         Market Buzz       0.81      0.93      0.86        27
       Negative News       0.81      0.93      0.86        27
              People       0.50      1.00      0.67         1
   Stock Performance       0.50      0.80      0.62         5
  Uncategorised News       0.50      0.50      0.50         2

           micro avg       0.77      0.85      0.81     31952
           macro avg       0.61      0.67      0.61     31952
        weighted avg       0.76    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1870: UserWarning: labels size, 2379, does not match size of target_names, 12
  .format(len(labels), len(target_names))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [41]:
# # Final Model: Multinomial Naive Bayesian
# **Multinomial Naive Bayesian** works the best. Lets run NB on our test data and get the confusion matrix and its heat map.
# ## Predict test data
print("\n\nPredicting test data using Multinomial Naive Bayesian")
pred_final = nb.predict(Xte)
print(classification_report(Yte, pred_final, labels=Ytr ,target_names=encoder.classes_))
print('Accuracy achieved is ' + str(np.mean(pred_final == Yte)))




Predicting test data using Multinomial Naive Bayesian
                      precision    recall  f1-score   support

Awards & Recognition       0.58      0.79      0.67        19
   Corporate Updates       0.50      0.33      0.40         6
               Event       0.00      0.00      0.00         4
              Events       0.00      0.00      0.00         2
           Financial       0.58      0.79      0.67        19
              Growth       0.58      0.79      0.67        19
       Industry News       0.58      0.79      0.67        19
         Market Buzz       0.58      0.79      0.67        19
       Negative News       0.58      0.79      0.67        19
              People       0.00      0.00      0.00         2
   Stock Performance       0.33      0.83      0.48         6
  Uncategorised News       0.00      0.00      0.00         2

           micro avg       0.53      0.69      0.60     25373
           macro avg       0.39      0.53      0.43     25373
        weig

In [47]:
def compareAlgo(x, y):
    print("Dummy cl")
    pred = dc.predict(x)
    print('Accuracy achieved is ' + str(np.mean(pred == y)))

    ### Decision Tree
    print("Decision Tree")
    pred = dt.predict(x)
    print('Accuracy achieved is ' + str(np.mean(pred == y)))


    # ### Random Forest
    print("Random Forest")
    pred = rf.predict(x)
    print('Accuracy achieved is ' + str(np.mean(pred_final == y)))

    ### Multinomial Naive Bayesian
    print("Training Multilayered Perceptron")
    pred_nb = nb.predict(x)
    print('Accuracy achieved is ' + str(np.mean(pred_final == y)))

    # ### Support Vector Classification
    print("Training Multilayered Perceptron")
    pred = svc.predict(x)
    print('Accuracy achieved is ' + str(np.mean(pred_final == y)))

    # ### Multilayered Perceptron
    print("Training Multilayered Perceptron")
    pred = mlp.predict(x)
    print('Accuracy achieved is ' + str(np.mean(pred_final == y)))


In [48]:
print("dev data")
compareAlgo(Xde, Yde)
print("test data")
compareAlgo(Xte, Yte)

dev data
Dummy cl
Accuracy achieved is 0.30612244897959184
Decision Tree
Accuracy achieved is 0.5102040816326531
Random Forest
Accuracy achieved is 0.0
Training Multilayered Perceptron
Accuracy achieved is 0.0
Training Multilayered Perceptron


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Accuracy achieved is 0.0
Training Multilayered Perceptron
Accuracy achieved is 0.0
test data
Dummy cl
Accuracy achieved is 0.16
Decision Tree
Accuracy achieved is 0.38
Random Forest
Accuracy achieved is 0.5
Training Multilayered Perceptron
Accuracy achieved is 0.5
Training Multilayered Perceptron
Accuracy achieved is 0.5
Training Multilayered Perceptron
Accuracy achieved is 0.5


In [45]:
pred_final = mlp.predict(Xte)
print('Accuracy achieved is ' + str(np.mean(pred_final == Yte)))
output = {"body":vectorizer.inverse_transform(Xte), "predicted":encoder.inverse_transform(pred_final),"actual": encoder.inverse_transform(Yte)}
df = pd.DataFrame(output, columns=["body","predicted","actual"])


Accuracy achieved is 0.5


In [43]:
df.to_csv("C:\\Users\\Kratika\\Downloads\\News333_body_predication.csv")